In [1]:
from __future__ import print_function
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist, cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split
from functools import partial

## Loading data

First, we load in the data and split it into a train and test set. 

In [2]:
# load the grayscale images and two-integer labels
images = np.load('images.npy')
labels = np.load('labels.npy')
images = images.astype('float32')
images /= 255

# input image dimensions
img_rows, img_cols = 150, 150

# reshape the 2D input data to 4D to fit the conv2D layer
if K.image_data_format() == 'channels_first':
    images = images.reshape(images.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    images = images.reshape(images.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# the data, split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (14400, 150, 150, 1)
14400 train samples
3600 test samples


## Definition of common-sense difference

The difference between the predicted time and actual time will be quantified using the 'common-sense' time. The function that calculates this time difference is shown below.

In [3]:
def common_sense(time1, time2):
    time1min = time1[0]*60 + time1[1]
    time2min = time2[0]*60 + time2[1]
    
    diff = abs(time1min - time2min)
    csdiff = min(diff, 12*60-diff)
    csdiffhr = np.array([int(csdiff/60), csdiff%60])
    
    return csdiff

print('Time 1: ', labels[500], '\nTime 2: ', labels[17500], '\nDifference in minutes: ', common_sense(labels[500], labels[17500]) )

Time 1:  [ 0 20] 
Time 2:  [11 40] 
Difference in minutes:  40


## Regression

In [4]:
# change the two-interger labels to a float
y_reg_train = y_train[:, 0] + y_train[:, 1] / 60
y_reg_test = y_test[:, 0] + y_test[:, 1] / 60

# the common sense accuracy for regression model
def common_sense_reg(y_true, y_pred):
    print(y_true, y_pred)
    differences = K.abs(y_pred - y_true)
    diff = tf.math.minimum(differences, tf.subtract(12.0, differences))
    return diff

batch_size = 128
epochs = 100

In [6]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 150, 150, 16)      160       
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 16)      2320      
                                                                 
 block1_conv3 (Conv2D)       (None, 150, 150, 16)      2320      
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 16)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 32)        4640      
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 32)        9248      
                                                                 
 block2_conv3 (Conv2D)       (None, 75, 75, 32)        9

In [5]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=(3, 3), activation="relu", padding="same")
model = keras.models.Sequential([
    DefaultConv2D(filters=16, input_shape=input_shape, name="block1_conv1"),
    DefaultConv2D(filters=16, name="block1_conv2"),
    DefaultConv2D(filters=16, name="block1_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block1_pool"),
    
    DefaultConv2D(filters=32, name="block2_conv1"),
    DefaultConv2D(filters=32, name="block2_conv2"),
    DefaultConv2D(filters=32, name="block2_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block2_pool"),
    
    DefaultConv2D(filters=64, name="block3_conv1"),
    DefaultConv2D(filters=64, name="block3_conv2"),
    DefaultConv2D(filters=64, name="block3_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block3_pool"),
    
    DefaultConv2D(filters=128, name="block4_conv1"),
    DefaultConv2D(filters=128, name="block4_conv2"),
    DefaultConv2D(filters=128, name="block4_conv3"),
    keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name="block4_pool"),

    keras.layers.Flatten(name="flatten"),
    keras.layers.Dense(1024, activation="relu", name="fc1"),
    keras.layers.Dense(32, activation="relu", name="fc2"),
    keras.layers.Dense(1, activation='linear', name="prediction"),
])

2022-11-17 16:43:29.138847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 16:43:29.150313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 16:43:29.150872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 16:43:29.155527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
model.compile(loss="mean_absolute_error", optimizer='sgd', metrics=[common_sense_reg])
# model.compile(loss="mean_absolute_error", optimizer='sgd', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.fit(X_train, y_reg_train, 
          batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_reg_test))
score = model.evaluate(X_test, y_reg_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2022-11-17 16:43:48.932774: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1296000000 exceeds 10% of free system memory.
2022-11-17 16:43:51.328933: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1296000000 exceeds 10% of free system memory.


Epoch 1/100
Tensor("ExpandDims_1:0", shape=(None, 1), dtype=float32) Tensor("sequential/prediction/BiasAdd:0", shape=(None, 1), dtype=float32)
Tensor("ExpandDims_1:0", shape=(None, 1), dtype=float32) Tensor("sequential/prediction/BiasAdd:0", shape=(None, 1), dtype=float32)


2022-11-17 16:43:56.091272: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


113/113 [==============================] - ETA: 0s - loss: 3.5565 - common_sense_reg: 3.0006

2022-11-17 16:44:06.438629: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 324000000 exceeds 10% of free system memory.
2022-11-17 16:44:07.073369: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 324000000 exceeds 10% of free system memory.


Tensor("ExpandDims_1:0", shape=(None, 1), dtype=float32) Tensor("sequential/prediction/BiasAdd:0", shape=(None, 1), dtype=float32)
113/113 [==============================] - 16s 79ms/step - loss: 3.5565 - common_sense_reg: 3.0006 - val_loss: 2.9746 - val_common_sense_reg: 2.9727
Epoch 2/100
113/113 [==============================] - 7s 62ms/step - loss: 3.0296 - common_sense_reg: 3.0025 - val_loss: 2.9933 - val_common_sense_reg: 2.9752
Epoch 3/100
113/113 [==============================] - 7s 64ms/step - loss: 3.0258 - common_sense_reg: 3.0009 - val_loss: 2.9742 - val_common_sense_reg: 2.9707
Epoch 4/100
113/113 [==============================] - 8s 67ms/step - loss: 3.0306 - common_sense_reg: 3.0124 - val_loss: 3.0100 - val_common_sense_reg: 2.9651
Epoch 5/100
113/113 [==============================] - 7s 65ms/step - loss: 3.0222 - common_sense_reg: 3.0090 - val_loss: 2.9974 - val_common_sense_reg: 2.9759
Epoch 6/100
113/113 [==============================] - 10s 87ms/step - loss: 3.0

2022-11-17 16:56:07.223307: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 324000000 exceeds 10% of free system memory.


Test loss: 2.9741859436035156
Test accuracy: 2.9705872535705566


In [80]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

553467096/553467096 [==============================] - 50s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

## Classification

In [8]:
# change the two-integer labels to 24 classes
def classification_label(y):
    label = []
    for i in y:
        if i[1] <= 30:
            label.append(2 * i[0])
        else:
            label.append(2 * i[0] + 1)
    return label
y_class_train = np.array(classification_label(y_train))
y_class_test = np.array(classification_label(y_test))

batch_size = 128
num_classes = 24 # maximum 720
epochs = 12

# convert class vectors to binary class matrices
y_class_train = keras.utils.to_categorical(y_class_train, num_classes)
y_class_test = keras.utils.to_categorical(y_class_test, num_classes)

In [9]:
# the common sense accuracy for classification model
def common_sense_class(y_true, y_pred):
    differences = K.abs(y_pred - y_true)
    diff = tf.math.minimum(differences, tf.subtract(float(num_classes), differences))
    return diff

In [10]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=(3, 3), activation="relu", padding="same")
model = keras.models.Sequential([
    DefaultConv2D(filters=16, input_shape=input_shape, name="block1_conv1"),
    DefaultConv2D(filters=16, name="block1_conv2"),
    DefaultConv2D(filters=16, name="block1_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block1_pool"),

    DefaultConv2D(filters=32, name="block2_conv1"),
    DefaultConv2D(filters=32, name="block2_conv2"),
    DefaultConv2D(filters=32, name="block2_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block2_pool"),
    
    DefaultConv2D(filters=64, name="block3_conv1"),
    DefaultConv2D(filters=64, name="block3_conv2"),
    DefaultConv2D(filters=64, name="block3_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block3_pool"),
    
    DefaultConv2D(filters=128, name="block4_conv1"),
    DefaultConv2D(filters=128, name="block4_conv2"),
    DefaultConv2D(filters=128, name="block4_conv3"),
    keras.layers.MaxPooling2D((2, 2), name="block4_pool"),

    keras.layers.Flatten(name="flatten"),
    keras.layers.Dense(1024, activation="relu", name="fc1"),
    keras.layers.Dense(64, activation="relu", name="fc2"),
    keras.layers.Dense(num_classes, activation="softmax", name="prediction"),
])

In [11]:
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
# model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=[common_sense_class])

model.fit(X_train, y_class_train,
        batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_class_test))
score = model.evaluate(X_test, y_class_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
113/113 [==============================] - 9s 75ms/step - loss: 3.1781 - accuracy: 0.0426 - val_loss: 3.1780 - val_accuracy: 0.0425
Epoch 2/12
113/113 [==============================] - 7s 66ms/step - loss: 3.1780 - accuracy: 0.0430 - val_loss: 3.1780 - val_accuracy: 0.0406
Epoch 3/12
113/113 [==============================] - 7s 64ms/step - loss: 3.1779 - accuracy: 0.0442 - val_loss: 3.1779 - val_accuracy: 0.0406
Epoch 4/12
113/113 [==============================] - 8s 70ms/step - loss: 3.1779 - accuracy: 0.0431 - val_loss: 3.1779 - val_accuracy: 0.0406
Epoch 5/12
113/113 [==============================] - 7s 64ms/step - loss: 3.1778 - accuracy: 0.0436 - val_loss: 3.1778 - val_accuracy: 0.0406
Epoch 6/12
113/113 [==============================] - 7s 66ms/step - loss: 3.1778 - accuracy: 0.0421 - val_loss: 3.1778 - val_accuracy: 0.0406
Epoch 7/12
113/113 [==============================] - 7s 64ms/step - loss: 3.1778 - accuracy: 0.0435 - val_loss: 3.1778 - val_accuracy: 0.0406

## Multi-head models

In [12]:
# change the two-integer labels to two categories
y_hours_train, y_minutes_train = np.split(y_train, 2, 1)
y_hours_test, y_minutes_test = np.split(y_test, 2, 1)

batch_size = 128
num_classes = 12
epochs = 12

# convert class vectors to binary class matrices
y_hours_train = keras.utils.to_categorical(y_hours_train, num_classes)
y_hours_test = keras.utils.to_categorical(y_hours_test, num_classes)

In [13]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=(3, 3), activation="relu", padding="same")

input = keras.layers.Input(shape=input_shape)
block1_conv1 = DefaultConv2D(filters=16)(input)
block1_conv2 = DefaultConv2D(filters=16)(block1_conv1)
block1_conv3 = DefaultConv2D(filters=16)(block1_conv2)
block1_pool = keras.layers.MaxPooling2D((2, 2))(block1_conv3)

block2_conv1 = DefaultConv2D(filters=32)(block1_pool)
block2_conv2 = DefaultConv2D(filters=32)(block2_conv1)
block2_conv3 = DefaultConv2D(filters=32)(block2_conv2)
block2_pool = keras.layers.MaxPooling2D((2, 2))(block2_conv3)

block3_conv1 = DefaultConv2D(filters=64)(block2_pool)
block3_conv2 = DefaultConv2D(filters=64)(block3_conv1)
block3_conv3 = DefaultConv2D(filters=64)(block3_conv2)
block3_pool = keras.layers.MaxPooling2D((2, 2))(block3_conv3)

block4_conv1 = DefaultConv2D(filters=128)(block3_pool)
block4_conv2 = DefaultConv2D(filters=128)(block4_conv1)
block4_conv3 = DefaultConv2D(filters=128)(block4_conv2)
block4_pool = keras.layers.MaxPooling2D((2, 2))(block4_conv3)

flatten = keras.layers.Flatten()(block4_pool)
fc1 = keras.layers.Dense(1024, activation="relu")(flatten)
fc2 = keras.layers.Dense(42, activation="relu")(fc1)

output1 = keras.layers.Dense(num_classes, activation="softmax")(fc2)
output2 = keras.layers.Dense(1, activation='linear')(fc2)
model = keras.models.Model(inputs=[input], outputs=[output1, output2])

In [14]:
model.compile(loss=["categorical_crossentropy", "mae"], optimizer="sgd", 
              metrics=["accuracy", "mean_squared_error"])

history = model.fit(X_train, (y_hours_train, y_minutes_train), batch_size=batch_size, epochs=epochs, verbose=1, 
                    validation_data=(X_test, (y_hours_test, y_minutes_test)))
score = model.evaluate(X_test, (y_hours_test, y_minutes_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# the dense 3 loss is rather high

Epoch 1/12
113/113 [==============================] - 10s 75ms/step - loss: 27.6714 - dense_2_loss: 2.7251 - dense_3_loss: 24.9463 - dense_2_accuracy: 0.0814 - dense_2_mean_squared_error: 0.0781 - dense_3_accuracy: 0.0165 - dense_3_mean_squared_error: 958.1273 - val_loss: 17.6891 - val_dense_2_loss: 2.5126 - val_dense_3_loss: 15.1765 - val_dense_2_accuracy: 0.0825 - val_dense_2_mean_squared_error: 0.0768 - val_dense_3_accuracy: 0.0181 - val_dense_3_mean_squared_error: 307.1940
Epoch 2/12
113/113 [==============================] - 8s 69ms/step - loss: 17.6665 - dense_2_loss: 2.5127 - dense_3_loss: 15.1538 - dense_2_accuracy: 0.0814 - dense_2_mean_squared_error: 0.0768 - dense_3_accuracy: 0.0163 - dense_3_mean_squared_error: 310.3399 - val_loss: 17.7354 - val_dense_2_loss: 2.5203 - val_dense_3_loss: 15.2151 - val_dense_2_accuracy: 0.0861 - val_dense_2_mean_squared_error: 0.0769 - val_dense_3_accuracy: 0.0181 - val_dense_3_mean_squared_error: 310.1121
Epoch 3/12
113/113 [=================

## Optimizing final model